# This is to scrape the HDB application rates data from various sources

In [12]:
# let's import some libraries

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service


In [ ]:
# Correct path to the ChromeDriver executable
service = Service('./chromedriver-win64/chromedriver.exe')  # Update the path if needed
driver = webdriver.Chrome(service=service)

In [10]:
# Open the Python website
driver.get("https://www.propertyguru.com.sg/property-guides/bto-application-rate-which-hdb-estate-easiest-62148")

In [17]:
element = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div[4]/div[1]/article/div[1]/figure[4]/div')

In [19]:
print(element.text)

Canberra Vista (Sembawang, Non-mature)
No. of units (3-, 4- and 5-Room)3-room: 124

4-room: 385

5-room/ 3Gen: 266



Total rooms:775
No. of applicants
3-room: 847
4-room: 3,048
5-room/ 3Gen: 3,738
= 7,633
Avg. application rate
3-room: 6.8
4-room: 7.9
5-room/ 3Gen: 14.1
= 9.8
Toa Payoh Ridge / Kim Keat Ripples (Toa Payoh, Mature)
No. of units (3-, 4- and 5-Room)
3-room: 102
4-room: 1,211
= 1,313
No. of applicants
3-room: 935
4-room: 11,684
= 12,619
Avg. application rate
3-room: 9.2
4-room: 9.6
= 9.6
